In [9]:
import numpy as np
from sortedcontainers import SortedSet
from math import sqrt, gcd
import random
import time
import matplotlib.pyplot as plt
import multiprocessing
import pickle

In [10]:
import pickle
with open('project-3-srcs_100.pkl', 'rb') as f:
    sources = pickle.load(f)
with open('project-3-tgts_100.pkl', 'rb') as f:
    targets = pickle.load(f)

In [11]:
class MyPriorityQueue(object):
    def __init__(self, current_heuristic, target):
        self.current_heuristic = current_heuristic
        self._data = SortedSet()
        self.target = target
        self.g = dict()
        self.h = dict()
    def __len__(self):
        return len(self._data)
    def push(self, item):
        node = (self.calculate_priority(item), self.get_heuristic(item), item)
        self._data.add(node)
    def pop(self):
        node = self._data.pop(0)[2]
        return node
    def manhattan_distance(self, x, y): ### Manhattan Distance-> heuristic = |x0-y0| + |x1-y1|, x = [x0,x1] y = [y0,y1]
        return abs(x[0] - y[0]) +abs(x[1] - y[1])
    def euclidean_distance(self, x, y): ### Euclidean Distance-> heuristic = sqrt( (x0-y0)**2 + (x1-y1)**2 ), x = [x0,x1]
                                        ##  y = [y0,y1]
        return sqrt((x[0] - y[0])**2 + (x[1] - y[1])**2)
    def chebyshev_distance(self, x, y): ### Chebyshev Distance-> heuristic = max(abs(x0-y0),abs(x1-y1)),
                                        ### x = [x0,x1] y = [y0,y1]
        return max(abs(x[0] - y[0]), abs(x[1] - y[1]))
    def get_heuristic(self, x):
        measure = self.current_heuristic
        if measure == 'm':
            self.h[x] = self.manhattan_distance(x, self.target)
        elif measure == 'e':
            self.h[x] = self.euclidean_distance(x, self.target)
        elif measure == 'c':
            self.h[x] = self.chebyshev_distance(x, self.target)
        return self.h[x]
    def calculate_priority(self, x):
        return self.g[x] + self.get_heuristic(x)

In [12]:
class GridWorld:
    def __init__(self, grid, start, target, n, m):
        '''
        Notations in grid: If grid contains '.', it means the cell is empty
                            and if the grid contains 'X', it means the cell is blocked
                            and if the grid contains 'P', it means the cell is of terrain type flat
                            and if the grid contains 'H', it means the cell is of terrain type hilly
                            and if the grid contains 'F', it means the cell is of terrain type forest
        '''
        self.n = n ## Number of columns
        self.m = m ## Number of rows
        self.start = start  ## Starting cell of the agent
        self.target = target ## Target cell of the agent
        self.trajectory = [] ## Trajectory followed by the agent
        self.grid = grid  ## Grid containing complete knowledge
        self.dirx8 = [0, 0, 1, -1, 1, -1, 1, -1] ## Directions used for sensing
        self.diry8 = [1, -1, 0, 0, 1, -1, -1, 1] ## Directions used for sensing
        self.dirx4 = [0, 0, 1, -1]  ## Directions used for traversing
        self.diry4 = [1, -1, 0, 0]  ## Directions used for traversing
            
        self.p_containing = {} #p_containing is the probability of (i,j) cell containing the target
        self.fn = {} #fn is the false negative of (i,j) cell 
        

In [16]:
class Agent9:
    def __init__(self, n, m, agent_grid, grid, start, target):
        self.n = n ## Number of columns
        self.m = m ## Number of rows
        self.agent_grid = agent_grid ## Current knowledge of the agent
        self.grid = grid ## Complete knowledge of the grid
        self.target = target ## The position from where the target starts moving
        self.start = start ## The position from where the agent starts looking for target
        self.assumed_target = None ## The position where the agent assumes the target is present
        self.fn = {'P':0.2,'H':0.5,'F':0.8}
        self.p_containing = np.ones((m,n))*(1/(n*m)) ## The initial belief of the agent
        self.prediction = np.ones((m,n))*(1/(n*m)) ## The initial prediction of the agent
        self.pre_vicinity = False
        self.dirx4 = [0, 0, -1, 1]
        self.diry4 = [-1, 1, 0, 0]
        self.dirx8 = [-1, 1, 0, 0, 1, -1, -1, 1]
        self.diry8 = [-1, 1, 1, -1, 0, 0, 1, -1]
        self.dist = np.zeros((m,n)) ## 2-d array used to store disance when calculating utility
        self.examine_cost =0
        self.movement_cost = 0
        self.examination_cost = 0
    def sense(self, curr_pos):
        '''
        The agent does the following things:
        1. Senses wheter the target is present in neighbourhood or not
        2. If the target is in the neighbourhood, updates the belief accordingly.
        3. If the target is not in the neighbourhood, update the belief accordingly.
        4. Returns vicinity variable, which stores whether the target is in the vicinity or not. 
        '''
        x = curr_pos[0]
        y = curr_pos[1]
        vicinity = False
        neighbouring_cell = []
        sum_prob = 0
        for i in range(8):
            next_x = x + self.dirx8[i]
            next_y = y + self.diry8[i]
            if( next_x >= 0 and next_x < self.m and next_y >= 0 and next_y < self.n and self.agent_grid.grid[next_x][next_y]!='X'):
                neighbouring_cell.append((next_x, next_y))
                sum_prob += self.p_containing[next_x][next_y]
                if self.target == (next_x, next_y):
                    vicinity = True
        total_prob = self.p_containing.sum()
        if vicinity == False:
            ##### For all the non-neighbouring cell, we need to update self.p_containing[cellx][celly] = self.p_containing/(total - sum_prob)
            for cellx in range(self.m):
                for celly in range(self.n):
                    cell = (cellx, celly)
                    if cell in neighbouring_cell:
                        self.p_containing[cellx][celly] = 0
                    else:
                        self.p_containing[cellx][celly] = self.p_containing[cellx][celly]/(total_prob - sum_prob)
        else:
            for cellx in range(self.m):
                for celly in range(self.n):
                    cell = (cellx, celly)
                    if cell in neighbouring_cell:
                        self.p_containing[cell[0]][cell[1]] = self.p_containing[cell[0]][cell[1]]/(sum_prob)
                    else:
                        self.p_containing[cell[0]][cell[1]] = 0
        return vicinity
    
    def update_probability_not_containing_target(self, curr_pos):
        cellx = curr_pos[0]
        celly = curr_pos[1]
        terrain = self.agent_grid.grid[cellx][celly]
        fn = self.fn[terrain]
        prob = self.p_containing[cellx][celly]
        total_prob = self.p_containing.sum()
        denominator = total_prob +(fn-1)*prob
        self.p_containing = self.p_containing/denominator
        self.p_containing[cellx][celly] = fn*prob/denominator
        
    def examination_cell(self, curr_pos):
        '''
        This function does the following things:
        1. Checks whether the target is present in the current position.
        2. If the target is not in the current position - returns False
        3. If the target is in the current position - returns True with the probabiliy of FN.
        4. return True or False, which indicates whether target is present in the cell.
        '''
        self.examination_cost +=1
        if self.target != curr_pos:
            self.update_probability_not_containing_target(curr_pos)
            return False
        else:
            cellx = curr_pos[0]
            celly = curr_pos[1]
            terrain = self.agent_grid.grid[cellx][celly]
            if random.random() < self.fn[terrain]:
                self.update_probability_not_containing_target(curr_pos)
                return False
            else:
                return True
            
    def generate_prediction_matrix(self):
        '''
        This function calculates the prediction matrix from the current knowledge of the agent.
        '''
        self.prediction = np.zeros((self.m,self.n))
        for cellx in range(self.m):
            for celly in range(self.n):
                cnt = 0
                if(self.agent_grid.grid[cellx][celly]=='X'):
                    continue
                for i in range(4):
                    nextx = cellx + self.dirx4[i]
                    nexty = celly + self.diry4[i]
                    if nextx >= 0 and nextx < self.m and nexty >= 0 and nexty < self.n and self.agent_grid.grid[nextx][nexty] != 'X':
                        cnt+=1
                for i in range(4):
                    nextx = cellx + self.dirx4[i]
                    nexty = celly + self.diry4[i]
                    if nextx >= 0 and nextx < self.m and nexty >= 0 and nexty < self.n and self.agent_grid.grid[nextx][nexty] != 'X':
                        self.prediction[nextx][nexty] = self.prediction[nextx][nexty] + self.p_containing[cellx][celly]/cnt
        return self.prediction
    
    def calculate_distance(self, curr_pos):
        from collections import deque
        self.dist = np.ones((self.m,self.n))*(self.m*self.n+1)
        visited = np.zeros((self.m,self.n))
        q = deque([])
        currx = curr_pos[0]
        curry = curr_pos[1]
        self.dist[currx][curry] = 0
        visited[currx][curry] = 1
        q.append(curr_pos)
        while( len(q)>0 ):
            cell = q.popleft()
            cellx = cell[0]
            celly = cell[1]
            for i in range(4):
                nextx = cellx + self.dirx4[i]
                nexty = celly + self.diry4[i]
                if nextx>=0 and nextx<self.m and nexty>=0 and nexty<self.n and self.agent_grid.grid[nextx][nexty] != 'X' and visited[nextx][nexty]==0:
                    visited[nextx][nexty] = 1
                    self.dist[nextx][nexty] = self.dist[cellx][celly] + 1
                    q.append((nextx, nexty))
        return self.dist
    
    def a_star(self, source, current_heuristic = 'm'):
        dirx = [-1,1,0,0]
        diry = [0,0,-1,1]
        visited = set()
        closed_list = set()
        open_list = MyPriorityQueue(current_heuristic, self.assumed_target)
        planned_path = []
        open_list.g[source] = 0
        open_list.calculate_priority(source)
        open_list.push(source)
        visited.add(source)
        parent = {}
        parent[source] = None
        while(len(open_list)>0):
            curr = open_list.pop()
            closed_list.add(curr)
            if curr[0] == self.assumed_target[0] and curr[1] == self.assumed_target[1]:
                break
            for i in np.arange(4):
                childx = curr[0] + dirx[i]
                childy = curr[1] + diry[i]
                child = (childx, childy)
                if(childx>=0 and childx<self.m and childy>=0 and childy<self.n and (child not in closed_list) and self.agent_grid.grid[childx][childy]!='X'):
                    if(child not in visited):
                        visited.add(child)
                        parent[child] = curr
                        open_list.g[child] = open_list.g[curr]+1
                        open_list.calculate_priority(child)
                        open_list.push(child)
                    elif open_list.g[curr]+1<open_list.g[child]:
                            parent[child] = curr
                            open_list._data.discard(child)
                            open_list.g[child] = open_list.g[curr]+1
                            open_list.calculate_priority(child)
                            open_list.push(child)
        if(self.assumed_target not in visited):
            return []
        curr = self.assumed_target
        while(curr != source):
            planned_path.append(curr)
            curr = parent[curr]
        planned_path.append(source)
        return planned_path[::-1]
    
    def update_probability_containing_block(self, block_pos):
        total_prob = self.p_containing.sum()
        x = block_pos[0]
        y = block_pos[1]
        prob = self.p_containing[x][y]
        self.p_containing = self.p_containing/(total_prob - prob)
        self.p_containing[x][y] = 0
        
    def move_target(self):
        neighbouring_cells = []
        currx = self.target[0]
        curry = self.target[1]
        for i in range(4):
            nextx = currx + self.dirx4[i]
            nexty = curry + self.diry4[i]
            if nextx>=0 and nextx<self.m and nexty>=0 and nexty<self.n and self.grid.grid[nextx][nexty] != 'X':
                neighbouring_cells.append((nextx,nexty))
        self.target = random.choice(neighbouring_cells)
        return self.target
    
    def pre_planning(self, curr_pos):
        '''
        This function calculates the next target with maximum utility based on the prediction
        '''
        ##### In this phase, the agent calculates the position of next target.
        ##### Calculate utility and get the new target
        self.calculate_distance(curr_pos)
        self.generate_prediction_matrix()
        assumed_target = None
        max_utility = 0
        max_utility_cells = []
        for i in range(self.m):
            for j in range(self.n):
                if i>=0 and i<self.m and j>=0 and j<self.n and self.agent_grid.grid[i][j]!='X' and self.dist[i][j]>0:
                    terrain = self.agent_grid.grid[i][j]
                    false_negative = self.fn.get(terrain,0)
                    curr_utility = (1-false_negative)*self.prediction[i][j]/self.dist[i][j]
                    if max_utility < curr_utility:
                        max_utility = curr_utility
                        max_utility_cells = [(i,j)]
                    elif max_utility == curr_utility:
                        max_utility_cells.append((i,j))
#         min_distance_cells = []
#         min_distance = self.m*self.n+1
#         for cell in max_utility_cells:
#             i = cell[0]
#             j = cell[1]
#             curr_dist = self.dist[i][j]
#             if curr_dist<min_distance:
#                 min_distance = curr_dist
#                 min_distance_cells = [(i,j)]
#             elif curr_dist == min_distance:
#                 min_distance_cells.append((i,j))
        self.assumed_target = random.choice(max_utility_cells)
        return self.assumed_target
    
    def planning(self, curr_pos):
        '''
        This function calculates the planned_path to the assumed target. It returns the path that is reachable to the assumed target. 
        '''
        ####### This function returns the next cell of the planned_path towards the assumed_target
        planned_path = self.a_star(curr_pos)
        while len(planned_path) == 0:
            assumed_target_x = self.assumed_target[0]
            assumed_target_y = self.assumed_target[1]
            self.agent_grid.grid[assumed_target_x][assumed_target_y] = 'X'
            self.update_probability_containing_block(self.assumed_target)
            self.assumed_target = self.pre_planning(curr_pos)
            planned_path = self.a_star(curr_pos)
        return planned_path
    
    def execution(self, curr_pos):
        ### In this function the agent will plan and travel along the path to the assumed target.
        '''
        This returns the next_cell of the agent until the next_cell is not blocked and there is a reachable planned_path to the assumed target.
        '''
        planned_path = self.planning(curr_pos)
        next_cell = planned_path[1]
        next_cellx = next_cell[0]
        next_celly = next_cell[1]
        while(self.grid.grid[next_cellx][next_celly] == 'X'):
            self.agent_grid.grid[next_cellx][next_celly] = 'X'
            self.update_probability_containing_block(self.assumed_target)
#             self.assumed_target = self.pre_planning(curr_pos)
            self.move_target()
            self.assumed_target = self.pre_planning(curr_pos)
            self.p_containing = self.generate_prediction_matrix()
            planned_path = self.planning(curr_pos)
            next_cell = planned_path[1]
            next_cellx = next_cell[0]
            next_celly = next_cell[1]
        return next_cell
    
    def find_target(self):
        curr_pos = self.start
        self.assumed_target = self.pre_planning(curr_pos)
        while True:
#             print(" curr_pos: ",curr_pos, " target: ",self.target," assumed_target: ",self.assumed_target)
            currx = curr_pos[0]
            curry = curr_pos[1]
            if self.agent_grid.grid[currx][curry] == '.':
                self.agent_grid.grid[currx][curry] = self.grid.grid[currx][curry]
            vicinity = self.sense(curr_pos) #### This function will sense the surrounding cells and find the target. If the cell finds the target, it ill change the 
                            #### probability accordingly. And if the cell does not fund the target, it will change the probability accordingly. It
                            #### will return whether the target is present in the vicinity or not.
#             self.assumed_target = self.pre_planning(curr_pos)
            if vicinity == True:
#                 self.assumed_target = self.pre_planning(curr_pos)
                self.move_target()
                self.p_containing = self.generate_prediction_matrix()
                self.pre_vicinity = True
                continue
            if self.pre_vicinity == True:
                self.pre_vicinity = False
                present = self.examination_cell(curr_pos) ### Tell whether target is present in curr_pos or not. IF present or not present,
#                 self.assumed_target = self.pre_planning(curr_pos)               ### the probabilities will update accordingly.
                if present:                               
                    print("Target found")
                    return
            self.assumed_target = self.pre_planning(curr_pos)
            next_cell = self.execution(curr_pos)
            curr_pos = next_cell 
            self.p_containing = self.generate_prediction_matrix()
            self.move_target()
            self.movement_cost +=1   
            
def multi_process(grid):
# for s in grid_data:
    # cnt = 0
#     print(grid[1])
    s = grid[0]
    start = grid[1]
    target = grid[2]
    # return 1,2,3,4
    result_y = []
    # print("Started for p: ",0.3)
    total_time = 0
    examine_cost = 0
    movement_cost = 0
    total_cost = 0
    # print(1)
    # s = grid
    i = 0
    grid = [] ## matrix that contains full knowledge of gridworld
    m = 100
    n = 100
    # start = sources
    # target = targets
    # print("Started for grid with source ", start, "and target ", target)
    while(i<m*n):
        grid.append(list(s[i:i+n]))
        i = i + n
    grid = np.array(grid)

    start_time = time.time()
    gridObject = GridWorld(grid, start, target, n, m) ### Original grid knowledge object
    agentGridObject = GridWorld(np.full((m,n),'.'), start, target, n, m) ### Agent Grid Object
    grid_agent = Agent9(n, m, agentGridObject, gridObject, start, target)
    ans = grid_agent.find_target()
#     ans = grid_agent.compute_path()
    end_time = time.time()
    
    total_time = end_time - start_time
    movement_cost = grid_agent.movement_cost
    examine_cost = grid_agent.examination_cost
    total_cost = grid_agent.movement_cost + grid_agent.examination_cost
    
    return total_time, movement_cost, examine_cost, total_cost

In [17]:
# def calculate_path_length(ans_compute_path):
#     ans = 0
#     for x in ans_compute_path:
#         ans = ans + len(x)-1
#     return ans + 1
# from tqdm import tqdm
# cnt = 0
# result_y = []
# print("Started for p: ",0.3)
# total_time = []
# examine_cost = []
# movement_cost = []
# total_cost = []
# avg_total_time = 0
# avg_examine_cost = 0
# avg_movement_cost = 0
# avg_total_cost = 0
# cnt = 0
# with open('./CS520-Assignment3/p0.3.txt') as f:
#     grid_data = f.readlines()
#     for s in grid_data:
#         i = 0
#         grid = [] ## matrix that contains full knowledge of gridworld
#         m = 50
#         n = 50
#         start = sources[cnt]
#         target = targets[cnt]
#         print("Started for grid ", cnt, "with source ", start, "and target ", target)
#         while(i<m*n):
#             grid.append(list(s[i:i+n]))
#             i = i + n
#         grid = np.array(grid)
#         start_time = time.time()
#         gridObject = GridWorld(grid, start, target, n, m) ### Original grid knowledge object
#         agentGridObject = GridWorld(np.full((m,n),'.'), start, target, n, m) ### Agent Grid Object
#         grid_agent = Agent9(n, m, agentGridObject, gridObject, start, target)
#         ans = grid_agent.find_target()
#         end_time = time.time()
#         print("Time taken for grid: ", end_time - start_time)
#         print("Movement Cost: ",self.movement_cost)
#         print("Examination Cost: ",self.examination_cost)
# #         #list of metrics
# #         total_time.append(end_time - start_time) #total time taken by the agent to find the target successfully
# #         movement_cost.append(calculate_path_length(ans)) #cost of traversing a cell by the agent
# #         examine_cost.append(grid_agent.examine_cost) #cost to examine a cell when the agent reaches on the 'assumed target'
# #         total_cost.append(calculate_path_length(ans) + grid_agent.examine_cost) #total cost = movement_cost + examine_cost
        
# #         #avg of metrics
# #         avg_total_time += end_time - start_time
# #         avg_movement_cost += calculate_path_length(ans)
# #         avg_examine_cost += grid_agent.examine_cost
# #         avg_total_cost += calculate_path_length(ans) + grid_agent.examine_cost
        
#         cnt+=1
#         if(cnt == 100):
#             break
            
#     print("Average Total Time: ", avg_total_time/100)
#     print("Average Examine Cost: ", avg_examine_cost/100)
#     print("Average Movement Cost: ", avg_movement_cost/100)
#     print("Average Total Cost: ", avg_total_cost/100)

In [18]:
# if __name__ == "__main__":
cores = int(multiprocessing.cpu_count())
# def multi_process():

cnt = 0
total_time = []
examine_cost = []
movement_cost = []
total_cost = []
avg_total_time = 0
avg_examine_cost = 0
avg_movement_cost = 0
avg_total_cost = 0
f = open('p0.3_100.txt', 'r')
grid_data = f.readlines()

p = multiprocessing.Pool(processes=cores)
for i in p.imap_unordered(multi_process,zip(grid_data,sources,targets)):
    cnt+=1
    print("Completed for grid:", cnt)
    print("Total Time: ", i[0], "Movement Cost: ", i[1], "Examine Cost: ", i[2], "Total Cost: ", i[3])
    total_time.append(i[0])
    movement_cost.append(i[1])
    examine_cost.append(i[2])
    total_cost.append(i[3])
    avg_total_time += i[0]
    avg_movement_cost += i[1]
    avg_examine_cost += i[2]
    avg_total_cost += i[3]
    if (cnt ==100):
        break

#     with open('Agent-9_avg_total_time.pkl', 'wb') as f:
#         pickle.dump(avg_total_time, f)
#     with open('Agent-9_avg_examine_cost.pkl', 'wb') as f:
#         pickle.dump(avg_examine_cost, f)
#     with open('Agent-9_avg_movement_cost.pkl', 'wb') as f:
#         pickle.dump(avg_movement_cost, f)
#     with open('Agent-9_avg_total_cost.pkl', 'wb') as f:
#         pickle.dump(avg_total_cost, f)
#     with open('Agent-9_total_time.pkl', 'wb') as f:
#         pickle.dump(total_time, f)
#     with open('Agent-9_examine_cost.pkl', 'wb') as f:
#         pickle.dump(examine_cost, f)
#     with open('Agent-9_movement_cost.pkl', 'wb') as f:
#         pickle.dump(movement_cost, f)
#     with open('Agent-9_total_cost.pkl', 'wb') as f:
#         pickle.dump(total_cost, f)
print("Average Total Time: ", avg_total_time/100)
print("Average Examine Cost: ", avg_examine_cost/100)
print("Average Movement Cost: ", avg_movement_cost/100)
print("Average Total Cost: ", avg_total_cost/100)

Target found
Completed for grid: 1
Total Time:  87.78992867469788 Movement Cost:  23 Examine Cost:  10 Total Cost:  33
Target found
Completed for grid: 2
Total Time:  283.2862980365753 Movement Cost:  210 Examine Cost:  2 Total Cost:  212
Target found
Completed for grid: 3
Total Time:  497.84935688972473 Movement Cost:  214 Examine Cost:  6 Total Cost:  220
Target found
Completed for grid: 4
Total Time:  549.5621020793915 Movement Cost:  246 Examine Cost:  6 Total Cost:  252
Target found
Completed for grid: 5
Total Time:  704.7302923202515 Movement Cost:  345 Examine Cost:  2 Total Cost:  347
Target found
Completed for grid: 6
Total Time:  706.6773567199707 Movement Cost:  471 Examine Cost:  12 Total Cost:  483
Target found
Completed for grid: 7
Total Time:  769.2894794940948 Movement Cost:  462 Examine Cost:  14 Total Cost:  476
Target found
Completed for grid: 8
Total Time:  951.5096526145935 Movement Cost:  545 Examine Cost:  1 Total Cost:  546
Target found
Completed for grid: 9
Tot

Target found
Completed for grid: 68
Total Time:  4864.087926626205 Movement Cost:  5391 Examine Cost:  27 Total Cost:  5418
Target found
Completed for grid: 69
Total Time:  4868.7518672943115 Movement Cost:  5589 Examine Cost:  5 Total Cost:  5594
Target found
Completed for grid: 70
Total Time:  4107.761120796204 Movement Cost:  4767 Examine Cost:  4 Total Cost:  4771
Target found
Completed for grid: 71
Total Time:  4932.127278804779 Movement Cost:  5587 Examine Cost:  1 Total Cost:  5588
Target found
Completed for grid: 72
Total Time:  4945.338797569275 Movement Cost:  5454 Examine Cost:  1 Total Cost:  5455
Target found
Completed for grid: 73
Total Time:  3369.115558862686 Movement Cost:  4068 Examine Cost:  2 Total Cost:  4070
Target found
Completed for grid: 74
Total Time:  5006.095621585846 Movement Cost:  5688 Examine Cost:  10 Total Cost:  5698
Target found
Completed for grid: 75
Total Time:  5070.811513900757 Movement Cost:  5786 Examine Cost:  12 Total Cost:  5798
Target found

In [19]:
with open('Agent-9_avg_total_time_100.pkl', 'wb') as f:
    pickle.dump(avg_total_time, f)
with open('Agent-9_avg_examine_cost_100.pkl', 'wb') as f:
    pickle.dump(avg_examine_cost, f)
with open('Agent-9_avg_movement_cost_100.pkl', 'wb') as f:
    pickle.dump(avg_movement_cost, f)
with open('Agent-9_avg_total_cost_100.pkl', 'wb') as f:
    pickle.dump(avg_total_cost, f)
with open('Agent-9_total_time_100.pkl', 'wb') as f:
    pickle.dump(total_time, f)
with open('Agent-9_examine_cost_100.pkl', 'wb') as f:
    pickle.dump(examine_cost, f)
with open('Agent-9_movement_cost_100.pkl', 'wb') as f:
    pickle.dump(movement_cost, f)
with open('Agent-9_total_cost_100.pkl', 'wb') as f:
    pickle.dump(total_cost, f)